In [17]:
#!/usr/bin/python
import os, sys
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
from io import StringIO
import re
from sklearn.model_selection import train_test_split
file = open("airports-extended.dat.txt","r")
import tensorflow as tf
from bokeh.plotting import figure, output_notebook, show


In [18]:

def powerlaw(x,a,b,c):
    return a*x**b+c

def f5(x,a,b,c,d,g,h):
    return a*x**5+b*x**4+c*x**3+d*x**2+g*x+h

def f3(x,a,b,c,d):
    return a*x**3+b*x**2+c*x+d


def VowelToConsRatio(name):
    
    words = sum(c.isalpha() for c in name)
    spaces = sum(c.isspace() for c in name)
    others = len(name) - words - spaces
    vowels = sum(map(name.lower().count, "aeiou"))
    consonents = words - vowels


    return float(vowels)/float(consonents), float(spaces)/float(consonents), float(others)/float(consonents)

def CharCount(name):
    
    words = sum(c.isalpha() for c in name)
    spaces = sum(c.isspace() for c in name)
    others = len(name) - words - spaces
    vowels = sum(map(name.lower().count, "aeiou"))
    consonents = words - vowels


    return int(consonents), int(vowels), int(spaces), int(others)



def MergeCount(totalCount, thisCount):
    totalCount2=totalCount+thisCount
    return totalCount2



def LatLongClassMaker(coor):
    lat,long=coor
    if lat<30:
        if long>-90:
            return 1 #florida
        elif long < -120:
            return 2 #hawaii
        else:
            return 3 #texas
    elif lat >50:
        return 4 #alaska
    elif long> -80 and lat < 40:
        return 5 #south east
    elif long> -80 and lat >40:
        return 6 #new england
    elif long< -80:
        return int((50-lat)/20.*5)*int((125+long)/45.*9)+7
    else:
        return 0
    #worst mapping ever-- ignores new england and city diversity
    return 0
ratio=[]
coords=[]



In [21]:
ratio=[]
coords=[]
charcountlist=[]
file = open("airports-extended.dat.txt","r")
for line in file:
    line=re.split(',',line);

    if line[3] == "\"United States\"":
        thisratioV, thisratioS, thisratioO=VowelToConsRatio(line[1]);
        ratio.append([thisratioV,thisratioS,thisratioO])
        charcountlist.append(np.array([CharCount(line[1])]))
        coords.append([float(line[6]),float(line[7])])


ratioarray=np.empty([len(ratio),3])
coordsarray=np.empty([len(coords),2])
regionarray=np.empty([len(coords)])
charcountarray=np.empty([len(coords),4])

for i, rat in enumerate(ratio):
    ratioarray[i,:]=rat
for i, charct in enumerate(charcountlist):
    charcountarray[i,:]=charct
for i,coor in enumerate(coords):
    coordsarray[i,:]=coor
#    outputcoor=LatLongClassMaker(coor)
#    regionarray[i]=outputcoor




In [22]:
testphrases=["Ufda!", "I just dont know about that", "Howdy, pardner", "Where are we?", "I thank whatever gods may be for my unconquerable soul.", "I welcome our new robot overlords", "l33t h4k3rs", "I know you I walked with you once upon a dream", "These stories dont mean anything if you have no one to tell them to", "Baby you have the sort of hands that rip me apart","Multi-messenger astronomy", "Rainbow flag", "I prefer They or He?", "numerical relativity", "LIGO", "scalar field", "Osculating Orbits", "Monte-Carlo Simulation", "Data Analysis", "Data Science", "Parallelization", "Paralyzation", "Partial disability", "Non-epileptic seizures", "Wednesday Lunch", "Tuesday Lunch", "Thursday Lunch", "Guild Wars", "Elvenar", "Good Apple"]


colleaguephrases=["general relativity", "black hole", "loop quantum gravity",  "quantization", "space-time", "Hamiltonian constraint", "Ashtekar", "LiSA", "LIGO", "group", "white hole", "scalar field", "numerical relativity", "cosmology", "diffeomorphism", "continuum limit", "David Berger", "Reisner-Nordstrom", "black hole spacetime", "initial data", "interpretation of quantum mechanics"]



In [5]:
xphrases=[]
for phrase in testphrases:
    print(phrase)
    vtc, stc, otc = VowelToConsRatio(phrase)
    xphrases.append([vtc,stc,otc])
xcolleague=[]
for phrase in colleaguephrases:
    print(phrase)
    vtc, stc, otc = VowelToConsRatio(phrase)
    xcolleague.append([vtc,stc,otc])

print(xphrases)




Ufda!
I just dont know about that
Howdy, pardner
Where are we?
I thank whatever gods may be for my unconquerable soul.
I welcome our new robot overlords
l33t h4k3rs
I know you I walked with you once upon a dream
These stories dont mean anything if you have no one to tell them to
Baby you have the sort of hands that rip me apart
Multi-messenger astronomy
Rainbow flag
I prefer They or He?
numerical relativity
LIGO
scalar field
Osculating Orbits
Monte-Carlo Simulation
Data Analysis
Data Science
Parallelization
Paralyzation
Partial disability
Non-epileptic seizures
Wednesday Lunch
Tuesday Lunch
Thursday Lunch
Guild Wars
Elvenar
Good Apple
general relativity
black hole
loop quantum gravity
quantization
space-time
Hamiltonian constraint
Ashtekar
LiSA
LIGO
group
white hole
scalar field
numerical relativity
cosmology
diffeomorphism
continuum limit
David Berger
Reisner-Nordstrom
black hole spacetime
initial data
interpretation of quantum mechanics
[[1.0, 0.0, 0.5], [0.5714285714285714, 0.357142

In [23]:
xtrain,xtest,ytrain,ytest=train_test_split(charcountarray,regionarray,test_size=0.33,shuffle=False, random_state=False)



In [24]:
trainshp=np.shape(ytrain)
testshp=np.shape(ytest)
trainshp[0]

1536

In [25]:
ytrain_flat = ytrain.flatten()
ytest_flat=ytest.flatten()

In [26]:
hidden_layers=20 #20 lines of division is my biased theory
hidden_size=3 #enough to draw a line with an offset I think, plus an little variance

In [27]:
sess = None

def reset_vars():
    sess.run(tf.global_variables_initializer())

def reset_tf():
    global sess
    if sess:
        sess.close()
    tf.reset_default_graph()
    sess = tf.Session()

In [11]:
len(xtrain)

1536

In [12]:
reset_tf()

In [13]:
x=xtrain

In [15]:
%%bash 
git add neuralnetwork.ipynb
git commit neuralnetwork.ipynb -m "first attempt at neural network"

[master f9441db] first attempt at neural network
 1 file changed, 443 insertions(+)
 create mode 100644 neuralnetwork.ipynb


In [14]:

W1 = tf.Variable(tf.random_normal([len(x), hidden_size], seed=42), name="weight1")
b1 = tf.Variable(tf.zeros([hidden_size]), name="bias1")
hidden = tf.matmul(x, W1) + b1
for i in np.arange(hidden_layers):

    W = tf.Variable(tf.random_normal([hidden_size, hidden_size]), name="weight"+str(i+2))
    b = tf.Variable(tf.random_normal([hidden_size]), name="bias"+str(i+2))
    hidden = tf.matmul(hidden, W) + b

W2 = tf.Variable(tf.random_normal([hidden_size, 1]), name="finalweight")
b2 = tf.Variable(tf.random_normal([1]), name="finalbias")
y = tf.matmul(hidden, W2) + b2
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=ytrain, 
                                                              labels=y_label))
train = tf.train.GradientDescentOptimizer(1e-1).minimize(loss)

predicted = tf.cast(tf.nn.sigmoid(ytrain) > 0.5, np.float32)
#accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,y_label),np.float32))


TypeError: Input 'b' of 'MatMul' Op has type float32 that does not match type float64 of argument 'a'.

In [ ]:
reset_vars()

for i in range(3000):
    sess.run(train,
             feed_dict={x: xtrain, y_label: ytrain})
    if i % 300 == 0:
        print sess.run([loss, accuracy], 
                       feed_dict={x: xtrain, y_label: ytrain}